In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from models import HuBERTEarlyFeatureFusion
import pandas as pd

import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

from torchview import draw_graph
import torchvision
# from torchviz import make_dot
from torchsummary import summary

# import torch
# torch.autograd.detect_anomaly(True)

/home/maximos/miniconda3/envs/torchaudio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_audio_folder = '/media/maindisk/maximos/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maindisk/maximos/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
feats['singer_id']

0      0
1      0
2      0
3      0
4      0
      ..
380    6
381    6
382    6
383    6
384    6
Name: singer_id, Length: 385, dtype: int64

In [5]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 7}


In [6]:
model = HuBERTEarlyFeatureFusion(task_labels_num_out=task_labels_num_out, gpu_index=1)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [8]:
print(len(training_data))

336


In [9]:
print(training_data[335])

{'input_values': array([ 6.63151741e-02,  6.36762679e-02,  1.43206418e-02,  1.13126114e-02,
       -1.97734162e-02, -3.63530293e-02, -3.67369354e-02, -3.97899300e-02,
       -6.59640953e-02, -1.10849649e-01, -1.65593401e-01, -1.67916626e-01,
       -1.29453331e-01, -1.03683472e-01, -1.15906730e-01, -1.50822371e-01,
       -1.81897819e-01, -1.89092189e-01, -1.70847028e-01, -1.38097763e-01,
       -1.24958776e-01, -1.20062985e-01, -1.22621045e-01, -1.09707057e-01,
       -9.05426145e-02, -6.76485151e-02, -3.79538499e-02, -1.14752566e-02,
        1.35118086e-02,  2.26801950e-02,  4.37688679e-02,  7.53356442e-02,
        1.07832104e-01,  1.27653271e-01,  1.41064093e-01,  1.52479336e-01,
        1.63866520e-01,  1.70812577e-01,  1.74290821e-01,  1.76015258e-01,
        1.81630239e-01,  1.89152926e-01,  1.81579292e-01,  1.42631471e-01,
        9.12469774e-02,  6.74288496e-02,  9.19982269e-02,  1.34408534e-01,
        1.27520263e-01,  5.10823876e-02, -4.92260419e-02, -9.59112942e-02,
       -

In [10]:
dataloader = DataLoader(training_data, batch_size=2, shuffle=True, collate_fn=model.collate_fn)

In [11]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x7b40e7427380>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 2, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x7b401b78fef0>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x7b40136b0320>, 'generator': None, 'collate_fn': <bound method HuBERTEarlyFeatureFusion.collate_fn of HuBERTEarlyFeatureFusion(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), 

In [12]:
b = next(iter(dataloader))

In [13]:
print(training_data.names[223])
print(training_data.feats['names'].str.contains('kazantzidisold').any())

kazantzidisold_w_10
True


In [14]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

2
2
33
{'Unnamed: 0': [302, 139], 'names': ['tsaousakis_a_13', 'kazantzidisold_e_4'], 'singer_id': [6, 3], 'Pitch': [0.036049760237776646, 0.1864611559634963], 'SpectralCentroid': [0.2795250799888653, 0.39263221346266125], 'SpectralSpread': [0.30939808960162807, 0.4824124261944179], 'SpectralSkewness': [0.5179794017964897, 0.30941813653296985], 'SpectralKurtosis': [0.2737389858085954, 0.0642965058457786], 'SpectralFlatness': [0.00012614140314942806, 0.04139388668788378], 'SpectralCrest': [0.038738561651637124, 0.1278219782760998], 'SpectralSlope': [0.4723857448760567, 0.4065613033400427], 'SpectralDecrease': [0.06820942338712029, 0.14352571594158778], 'SpectralRollOff': [0.28085338577084235, 0.23044024138785943], 'SpectralVariation': [0.5353453846942953, 0.7057184582205537], 'SpectralFlux': [0.0902278912045978, 0.1881036030101136], 'HarmonicSpectralDeviation': [0.07820715420157692, 0.1300031099861007], 'Tristimulus_1': [0.04671876787718817, 0.3620174193328774], 'Tristimulus_2': [0.2725

In [15]:
model.train()
# model.eval()
for p in model.hubert.parameters():
    p.requires_grad = False

model.hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0.requires_grad = True
# model.hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1.requires_grad = True
# train_params = []
# train common base
# model.common_base_1.requires_grad = True
# model.common_base_2.requires_grad = True
model.common_projector.requires_grad = True
for k in model.projectors.keys():
    # train_params.extend( list(model.projectors[k].parameters()) )
    # train_params.extend( list(model.classifiers[k].parameters()) )
    print(k)
    model.intermediates[k].requires_grad = True
    model.projectors[k].requires_grad = True
    model.classifiers[k].requires_grad = True
# # model.train(False)
# print(train_params)
# optimizer = Adam( model.classifiers['Pitch'].parameters() )
optimizer = Adam( model.parameters(), lr=0.001 )

Pitch
SpectralCentroid
SpectralSpread
SpectralSkewness
SpectralKurtosis
SpectralFlatness
SpectralCrest
SpectralSlope
SpectralDecrease
SpectralRollOff
SpectralVariation
SpectralFlux
HarmonicSpectralDeviation
Tristimulus_1
Tristimulus_2
Tristimulus_3
HarmonicOddToEvenRatio
Inharmonicity
HarmonicEnergy
NoiseEnergy
Noisiness
HarmonicToNoiseEnergy
PartialsToNoiseEnergy
F1_Hz
F2_Hz
F3_HZ
F4_Hz
singer_id


In [16]:
for p in model.parameters():
    print(p)
# print(model.parameters())

Parameter containing:
tensor([1.2553e-02, 1.4869e-01, 1.9549e-01, 3.2281e-01, 9.8338e-01, 5.8375e-01,
        8.7520e-01, 1.5427e-01, 2.7129e-01, 3.6664e-01, 1.6674e-01, 1.7845e-01,
        5.5391e-01, 3.9156e-01, 1.6317e-01, 6.6617e-01, 3.0099e-01, 3.2494e-01,
        4.3132e-02, 6.0423e-01, 1.6720e-02, 7.2768e-01, 4.5652e-01, 2.7838e-01,
        9.8475e-02, 1.8306e-01, 6.5849e-01, 8.3431e-01, 4.1389e-01, 3.4493e-01,
        3.6529e-01, 3.3995e-01, 9.8168e-01, 7.2934e-01, 2.4049e-01, 8.6519e-01,
        1.4189e-02, 5.9771e-02, 2.3626e-01, 7.6411e-01, 8.8876e-01, 2.7424e-01,
        5.5505e-01, 6.5420e-01, 5.6316e-01, 2.1844e-01, 5.4473e-02, 5.6561e-01,
        6.4149e-01, 1.9862e-02, 3.4729e-01, 4.8586e-01, 1.7547e-02, 4.7509e-01,
        9.0136e-01, 8.4827e-01, 7.7520e-01, 6.0375e-01, 7.7504e-01, 6.4502e-01,
        5.8461e-01, 7.7612e-01, 9.4040e-01, 4.6441e-01, 6.1678e-01, 6.6889e-01,
        8.4742e-01, 4.2614e-01, 9.1733e-01, 8.7176e-01, 3.2658e-01, 7.9176e-01,
        9.3804e-01

In [17]:
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [18]:
# print(model.hubert.encoder.layers[0].attention.k_proj.weight)
# print(model.common_base_1.weight)
print( model.hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1.shape )

torch.Size([768, 48, 128])


In [19]:
b[0]['attention_mask']

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [20]:
print(b[1]['singer_id'])

[6, 3]


In [21]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

In [22]:
print(y)

SequenceClassifierOutput(loss=tensor(4.3663, device='cuda:1', grad_fn=<AddBackward0>), logits={'Pitch': [tensor([[0.4812],
        [0.5155]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralCentroid': [tensor([[0.5229],
        [0.5381]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralSpread': [tensor([[0.5081],
        [0.5162]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralSkewness': [tensor([[0.5238],
        [0.4915]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralKurtosis': [tensor([[0.4761],
        [0.4955]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralFlatness': [tensor([[0.5173],
        [0.5162]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralCrest': [tensor([[0.4871],
        [0.4593]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralSlope': [tensor([[0.4460],
        [0.4823]], device='cuda:1', grad_fn=<SigmoidBackward0>)], 'SpectralDecrease': [tensor([[0.5592],
        [0.5236]], device='cuda:1', grad_fn=<

In [23]:
print(y['logits'].keys())
for k in y['logits'].keys():
    print(y['logits'][k])

dict_keys(['Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'singer_id'])
[tensor([[0.4812],
        [0.5155]], device='cuda:1', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5229],
        [0.5381]], device='cuda:1', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5081],
        [0.5162]], device='cuda:1', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5238],
        [0.4915]], device='cuda:1', grad_fn=<SigmoidBackward0>)]
[tensor([[0.4761],
        [0.4955]], device='cuda:1', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5173],
        [0.5162]], device='cuda:1', grad_fn=<SigmoidBackward0>)]

In [24]:
optimizer.zero_grad()
y.loss.backward()# retain_graph=True
optimizer.step()

Could not load library libcudnn_cnn_train.so.8. Error: /home/maximos/miniconda3/lib/python3.10/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /home/maximos/miniconda3/lib/python3.10/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /home/maximos/miniconda3/lib/python3.10/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /home/maximos/miniconda3/lib/python3.10/site-packages/nvidia/cudnn/lib/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn

RuntimeError: GET was unable to find an engine to execute this computation

In [26]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.__path__)

2.2.1+cu121
True
12.1
8900
['/home/maximos/miniconda3/envs/torchaudio/lib/python3.12/site-packages/torch']


In [ ]:
print(model)

HuBERTEarlyFeatureFusion(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEm

In [ ]:
# print(model.intermediates['Pitch'].weight.grad)
print(model.common_base_1.weight.grad)
# print(model.projectors['F4_Hz'].weight)

tensor([[-4.7804e-07,  8.5698e-06,  1.4979e-04,  ..., -1.2976e-06,
         -9.5237e-07,  1.2865e-03],
        [-1.7132e-07, -5.8099e-07,  5.1766e-05,  ..., -2.7368e-07,
         -2.5174e-07,  3.3459e-04],
        [ 1.5944e-07, -9.4141e-06, -5.3402e-05,  ...,  7.7625e-07,
          4.7843e-07, -6.5612e-04],
        ...,
        [-5.7301e-10,  1.1940e-07,  2.3684e-07,  ..., -7.2725e-09,
         -3.8179e-09,  5.3211e-06],
        [ 5.0473e-10, -1.1135e-07, -2.1186e-07,  ...,  6.7297e-09,
          3.5145e-09, -4.9010e-06],
        [-6.3529e-08, -9.2645e-06,  1.4445e-05,  ...,  3.7260e-07,
          1.2859e-07, -1.8929e-04]], device='cuda:0')


In [ ]:
print(model.hubert.encoder.layers[11].final_layer_norm.weight.grad)
print(model.hubert.encoder.layers[11].final_layer_norm.weight)

None
Parameter containing:
tensor([0.2295, 0.3408, 0.2241, 0.2180, 0.3718, 0.2739, 0.4177, 0.7271, 0.2072,
        0.2150, 0.2554, 0.2473, 0.2111, 0.7178, 0.1851, 0.2050, 0.4814, 0.2057,
        0.2379, 0.1952, 0.5957, 0.3611, 0.2522, 0.5278, 0.1882, 1.0547, 0.2152,
        0.4490, 0.4299, 0.2314, 0.2019, 0.2072, 0.2798, 0.2083, 0.4707, 0.2754,
        0.2136, 0.6519, 0.2372, 0.2583, 0.2527, 0.2141, 0.2001, 0.3301, 0.2196,
        0.2537, 0.2581, 0.2908, 0.2773, 0.2332, 0.1733, 0.2766, 0.2544, 0.2703,
        0.2329, 0.2493, 0.2891, 0.2959, 0.2852, 0.6724, 0.2040, 0.2484, 0.2087,
        0.3169, 0.2595, 0.1816, 0.2347, 0.7002, 0.2201, 0.2742, 0.2114, 0.2729,
        0.6841, 0.6235, 0.4590, 0.1656, 0.2288, 0.4587, 0.3438, 0.2225, 0.4719,
        0.2688, 0.2349, 0.2260, 0.2186, 0.2642, 0.2301, 0.3638, 0.2355, 0.4534,
        0.1997, 0.1818, 0.1923, 0.2277, 0.7339, 0.2678, 0.2124, 0.2478, 0.2356,
        0.3901, 0.2098, 0.2205, 0.2212, 0.6611, 0.2272, 0.2209, 0.2822, 0.2277,
        0.570

In [ ]:
print(len(model.hubert.encoder.layers))

12


In [ ]:
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight.grad)
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight)
# print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None
Parameter containing:
tensor([[-0.0309, -0.0560, -0.0944,  ..., -0.0551,  0.0054, -0.1141],
        [-0.0452,  0.0558, -0.0271,  ..., -0.0301, -0.0216,  0.0928],
        [ 0.0855, -0.0440,  0.1083,  ..., -0.0047,  0.0585, -0.1077],
        ...,
        [-0.0461, -0.0809,  0.0314,  ...,  0.0263,  0.0117,  0.0425],
        [-0.0258, -0.0419, -0.0305,  ..., -0.0015, -0.0509, -0.1317],
        [-0.0070, -0.0095,  0.0326,  ...,  0.0144,  0.0219, -0.0197]],
       device='cuda:0')


In [ ]:
for i in range(50):
    y = model(
        audio_normalized=b[0]['input_values'],
        attention_mask=b[0]['attention_mask'],
        labels=b[1],
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True
    )
    print(y.loss)
    # print(model.classifiers['F4_Hz'].weight)
    optimizer.zero_grad()
    y.loss.backward()
    optimizer.step()

tensor(3.7831, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2644, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7722, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4376, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2265, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1175, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0295, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9216, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8184, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7184, device='cuda:0', grad_fn=<AddBackward0>)


tensor(1.6555, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5979, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5495, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4796, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4130, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3952, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3545, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3241, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2803, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2705, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2499, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2371, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2325, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2129, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2202, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2086, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2011, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2069, device='cuda:0', grad_fn=<AddBack

In [ ]:
print(y['logits']['singer_id'][0].argmax(dim=1, keepdim=True).squeeze())

tensor([6, 2, 4, 5], device='cuda:0')
